In [1]:
import pandas as pd

# Khai báo nhập vào ngày bắt đầu và kết thúc
start_date = input("Nhập vào ngày bắt đầu (YYYYMMDD): ")
end_date = input("Nhập vào ngày kết thúc (YYYYMMDD): ")

# Chuyển đổi ngày bắt đầu và kết thúc sang kiểu datetime
start_datetime = pd.to_datetime(start_date, format='%Y%m%d')
end_datetime = pd.to_datetime(end_date, format='%Y%m%d')

# Lặp qua các ngày từ ngày bắt đầu đến ngày kết thúc
for date in pd.date_range(start=start_datetime, end=end_datetime):
    date_str = date.strftime('%Y%m%d')  # chuyển đổi datetime sang chuỗi
    print(f"Processing data for date {date_str}")

    df1 = pd.read_csv(f'{date_str}_1.csv')
    df2 = pd.read_csv(f'{date_str}_2.csv')

    # Ghép bảng
    merged_df = pd.merge(df1, df2[['user_pseudo_id','event_date','event_timestamp','event_name', 'engagement_time_msec']], on=(['user_pseudo_id', 'event_date','event_name', 'event_timestamp']), how = 'left')
    merged_df['engagement_time_msec'].fillna(value='null', inplace=True)

    # Xoá dòng lặp
    merged_df.drop_duplicates(inplace=True)
    
    # Xuất file
    event_file = f'event_{date_str}.csv'
    output_folder = r"C:\Users\ADMIN\Documents\Work\Ragdoll Dismounting Playground\Datacleansing"
    output_file = f"{output_folder}/{event_file}"
    merged_df.to_csv(output_file, index=False)

print("Done!")


Nhập vào ngày bắt đầu (YYYYMMDD):  20230326
Nhập vào ngày kết thúc (YYYYMMDD):  20230326


Processing data for date 20230326
Done!


In [9]:
import os
import pandas as pd

# Đường dẫn đến thư mục chứa các file CSV
folder_path = r"C:\Users\ADMIN\Documents\Work\Ragdoll Dismounting Playground\Datacleansing"

# Lấy danh sách tên các file CSV trong thư mục
file_names = os.listdir(folder_path)

# Lọc ra các file có tên dạng "event_YYYYMMDD.csv"
file_names = [file_name for file_name in file_names if file_name.startswith('event_') and file_name.endswith('.csv')]

# Định nghĩa hàm để đọc dữ liệu từ các tệp CSV
def read_csv_file(file_path):
    df = pd.read_csv(file_path)
    df["event_date"] = pd.to_datetime(df["event_date"])
    return df

# Đọc dữ liệu từ các tệp và tính toán số lượng user_id duy nhất trong mỗi bảng record
dfs = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name) # Đường dẫn đến file CSV
    df = read_csv_file(file_path)
    date_str = file_name.split("_")[1].split(".")[0] # Lấy ngày từ tên tệp
    df["event_date"] = date_str
    df_agg = df.groupby("event_date").agg({"user_pseudo_id": pd.Series.nunique})
    dfs.append(df_agg)

# Kết hợp các bảng kết quả
result = pd.concat(dfs)

# Xuất ra file csv trong một thư mục chỉ định
output_folder_path = r"C:\Users\ADMIN\Documents\Work\Ragdoll Dismounting Playground\Addition tables"
output_file_path = os.path.join(output_folder_path, 'active_user.csv')
result.to_csv(output_file_path, index=True)

print("Done!")

Done!
